#### last checkpoint with code change: 8/23/2022

Indicators of interest:
-	Percentage of population within 2h of driving to the nearest primary care facility (population level, and by SES quintile).
-	Percentage of population within 2h of driving to the nearest district hospital (population, and by SES quintile).
-	Percentage of health facilities having direct access to an all season road.
-	Percentage of health facilities within 2km of an all season road.

In [129]:
import os, sys

In [130]:
sys.path.append('/home/wb411133/Code/gostrocks/src') # gostrocks is used for some basic raster operations (clip and standardize)

In [131]:
sys.path.append('/home/wb411133/Code/GOSTNets_Raster/src') # gostnets_raster has functions to work with friction surface

In [132]:
sys.path.append('/home/wb411133/Code/GOSTnets') # it also depends on gostnets for some reason

In [133]:
sys.path.append('/home/wb411133/Code/INFRA_SAP') # only used to save some raster results
# sys.path.append('/home/wb514197/Repos/HospitalAccessibility/src') # only used to save some raster results

In [134]:
import GOSTRocks.rasterMisc as rMisc

In [135]:
import GOSTNetsRaster.market_access as ma

In [136]:
#import sys
#!{sys.executable} -m pip install rasterstats

In [137]:
from infrasap import aggregator
from rasterstats import zonal_stats

## Import Fiona (GOSTRocks.rasterMisc as rMisc) before geopandas to fix Fiona circular import issue. https://github.com/Toblerity/Fiona/issues/944

In [138]:
import geopandas as gpd
import pandas as pd
import rasterio as rio
import numpy as np
from shapely.geometry import Point
import skimage.graph as graph

In [139]:
iso3 = 'LBR'

In [140]:
input_dir = "/home/public/Data/PROJECTS/Health" #
out_folder = os.path.join(input_dir, "output", iso3)
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

In [141]:
global_admin2 = '/home/public/Data/GLOBAL/ADMIN/Admin2_Polys.shp'
adm2 = gpd.read_file(global_admin2)
adm2 = adm2.loc[adm2.ISO3==iso3].copy()
adm2 = adm2.to_crs("EPSG:4326")

In [142]:
adm2.reset_index(inplace=True)

In [143]:
sample_admin = os.path.join(input_dir,'output','LBR', 'sample_aoi_monrovia.shp')
sample_admin_gpd = gpd.read_file(sample_admin)

In [144]:
sample_admin_gpd

,id,geometry
0,1,"POLYGON ((-10.68870 6.52571, -10.44156 6.52647..."


In [145]:
out_pop_surface_std = os.path.join(out_folder, "WP_2020_1km_STD.tif")
pop_surf = rio.open(out_pop_surface_std)
pop = pop_surf.read(1, masked=True)

In [146]:
sub_category_list = ["health",
"hospital",
"offer_anc", 
"offer_emonc",
"offer_childimmune", 
"offer_essentialnutrition", 
"offer_adolescentsrhr"
]

In [147]:
zs_pop = pd.DataFrame(zonal_stats(adm2, pop.filled(), affine=pop_surf.transform, stats='sum', nodata=pop_surf.nodata)).rename(columns={'sum':'pop'})

In [148]:
#zs_pop

In [149]:
for index, sub_category in enumerate(sub_category_list):
    if index == 0:
        tt_rio = rio.open(os.path.join(out_folder, "tt_"+sub_category+"_2nd_rd.tif"))
        tt = tt_rio.read(1, masked=True)

        pop_120 = pop*(tt<=120)

        zs_lt_120 = pd.DataFrame(zonal_stats(adm2, pop_120.filled(), affine=pop_surf.transform, stats='sum', nodata=pop_surf.nodata)).rename(columns={'sum':'pop_120_'+sub_category})
        zs = zs_pop.join(zs_lt_120)
        zs.loc[:, sub_category+"_pct"] = zs.loc[:, "pop_120_"+sub_category]/zs.loc[:, "pop"]
        res = adm2.join(zs)
    else:
        tt_rio = rio.open(os.path.join(out_folder, "tt_"+sub_category+"_2nd_rd.tif"))
        tt = tt_rio.read(1, masked=True)

        pop_120 = pop*(tt<=120)

        zs_lt_120 = pd.DataFrame(zonal_stats(adm2, pop_120.filled(), affine=pop_surf.transform, stats='sum', nodata=pop_surf.nodata)).rename(columns={'sum':'pop_120_'+sub_category})

        zs = zs.join(zs_lt_120)
        zs.loc[:, sub_category+"_pct"] = zs.loc[:, "pop_120_"+sub_category]/zs.loc[:, "pop"]
        res = adm2.join(zs)

In [150]:
res.to_file(os.path.join(out_folder, "pop_within_2hrs_sub_categories.shp"), driver="ESRI Shapefile")

/tmp/ipykernel_44758/3313369315.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  res.to_file(os.path.join(out_folder, "pop_within_2hrs_sub_categories.shp"), driver="ESRI Shapefile")


## Roads

#### Import and filter hospital points

In [151]:
import json
from utm_zone import epsg as epsg_get

In [152]:
adm2_json = json.loads(adm2.to_json())
epsg = epsg_get(adm2_json)

In [153]:
epsg

32629

In [154]:
# round 2 way
lbr_master = pd.read_csv(os.path.join(input_dir, "from_tashrik", "master lists", "Liberia MFL Adjusted.csv"), index_col=0)
lbr_geocoded = pd.read_csv(os.path.join(input_dir, "from_tashrik", "master lists", "Liberia MFL Adjusted Geocoded_Validated.csv"), index_col=0)
lbr = lbr_master.loc[lbr_master.functional=="functional"].copy()
lbr = lbr.loc[~(lbr.latitude.isna())].copy()
#len(lbr), len(lbr_geocoded)
lbr_geocoded = lbr_geocoded.loc[lbr_geocoded.geocoding_method!="District name and county query"].copy()
#len(lbr), len(lbr_geocoded)
lbr = pd.concat([lbr, lbr_geocoded])
lbr.reset_index(drop=True, inplace=True)
#lbr.columns
#lbr.facilitytype.value_counts()
geoms = [Point(xy) for xy in zip(lbr.longitude, lbr.latitude)]
lbr_geo = gpd.GeoDataFrame(lbr, crs='EPSG:4326', geometry=geoms)

In [155]:
global_admin = '/home/public/Data/GLOBAL/ADMIN/g2015_0_simplified.shp'
adm0 = gpd.read_file(global_admin)
aoi = adm0.loc[adm0.ISO3166_1_==iso3]
lbr_geo_filt = lbr_geo.loc[lbr_geo.intersects(aoi.unary_union)]
lbr_geo_filt.reset_index(inplace=True, drop=True)
hospitals = lbr_geo_filt.loc[lbr_geo_filt.facilitytype=="hospital"].copy()
hospitals.reset_index(inplace=True, drop=True)
print(lbr_geo_filt[['facilitytype']].value_counts())
print(len(lbr_geo_filt), len(hospitals))
lbr_geo_filt.to_file(os.path.join(out_folder, "LBR_health.geojson"), driver='GeoJSON')
lbr_geo_filt = lbr_geo_filt.to_crs(epsg)
hospitals = hospitals.to_crs(epsg)

facilitytype 
clinic           735
health center     87
hospital          24
health post        6
schoold based      4
SDP                1
dtype: int64
857 24


In [156]:
from infrasap import osm_extractor as osm

In [157]:
import json
from utm_zone import epsg as epsg_get

In [158]:
roads = gpd.read_file(os.path.join(input_dir, 'osm', 'highways.shp'))

In [159]:
adm2_json = json.loads(adm2.to_json())
epsg = epsg_get(adm2_json)

In [160]:
epsg

32629

In [161]:
roads = roads.to_crs(epsg)

In [162]:
def get_num(x):
    try:
        return(int(x))
    except:
        return(5)

In [163]:
osm.OSMLR_Classes

{'motorway': 'OSMLR level 1',
 'motorway_link': 'OSMLR level 1',
 'trunk': 'OSMLR level 1',
 'trunk_link': 'OSMLR level 1',
 'primary': 'OSMLR level 1',
 'primary_link': 'OSMLR level 1',
 'secondary': 'OSMLR level 2',
 'secondary_link': 'OSMLR level 2',
 'tertiary': 'OSMLR level 2',
 'tertiary_link': 'OSMLR level 2',
 'unclassified': 'OSMLR level 3',
 'unclassified_link': 'OSMLR level 3',
 'residential': 'OSMLR level 3',
 'residential_link': 'OSMLR level 3',
 'track': 'OSMLR level 4',
 'service': 'OSMLR level 4'}

In [164]:
sub_category_list = ["offer_anc", 
"offer_emonc",
"offer_childimmune", 
"offer_essentialnutrition", 
"offer_adolescentsrhr"
]

In [165]:
def compute_facilities_within_buffer_of_roads(roads, buffers = [100, 2000], sub_category = None):
    roads_copy_dict = {}
    for buffer in buffers:
        roads_copy_dict[buffer] = roads.copy()
        roads_copy_dict[buffer]['geometry'] = roads_copy_dict[buffer]['geometry'].apply(lambda x: x.buffer(buffer))
        roads_copy_dict[buffer]['OSMLR'] = roads_copy_dict[buffer]['type'].map(osm.OSMLR_Classes)
        roads_copy_dict[buffer]['OSMLR_num'] = roads_copy_dict[buffer]['OSMLR'].apply(lambda x: get_num(str(x)[-1]))
        
        if sub_category is not None:
            lbr_geo_filt.loc[:, "bool_1_"+str(buffer)+"_"+sub_category] = lbr_geo_filt.loc[lbr_geo_filt[sub_category]==1].intersects(roads_copy_dict[buffer].loc[roads_copy_dict[buffer].OSMLR_num<=1].unary_union)
            lbr_geo_filt.loc[:, "bool_2_"+str(buffer)+"_"+sub_category] = lbr_geo_filt.loc[lbr_geo_filt[sub_category]==1].intersects(roads_copy_dict[buffer].loc[roads_copy_dict[buffer].OSMLR_num<=2].unary_union)
            lbr_geo_filt.loc[:, "bool_3_"+str(buffer)+"_"+sub_category] = lbr_geo_filt.loc[lbr_geo_filt[sub_category]==1].intersects(roads_copy_dict[buffer].loc[roads_copy_dict[buffer].OSMLR_num<=3].unary_union)
            lbr_geo_filt.loc[:, "bool_4_"+str(buffer)+"_"+sub_category] = lbr_geo_filt.loc[lbr_geo_filt[sub_category]==1].intersects(roads_copy_dict[buffer].loc[roads_copy_dict[buffer].OSMLR_num<=4].unary_union)
        else:
            lbr_geo_filt.loc[:, "bool_1_"+str(buffer)] = lbr_geo_filt.intersects(roads_copy_dict[buffer].loc[roads_copy_dict[buffer].OSMLR_num<=1].unary_union)
            lbr_geo_filt.loc[:, "bool_2_"+str(buffer)] = lbr_geo_filt.intersects(roads_copy_dict[buffer].loc[roads_copy_dict[buffer].OSMLR_num<=2].unary_union)
            lbr_geo_filt.loc[:, "bool_3_"+str(buffer)] = lbr_geo_filt.intersects(roads_copy_dict[buffer].loc[roads_copy_dict[buffer].OSMLR_num<=3].unary_union)
            lbr_geo_filt.loc[:, "bool_4_"+str(buffer)] = lbr_geo_filt.intersects(roads_copy_dict[buffer].loc[roads_copy_dict[buffer].OSMLR_num<=4].unary_union)

    #return roads_copy_dict
        

In [166]:
for sub_category in sub_category_list:
    compute_facilities_within_buffer_of_roads(roads, buffers = [100,2000], sub_category=sub_category)

# do for all facilities
compute_facilities_within_buffer_of_roads(roads, buffers = [100,2000], sub_category=None)

In [167]:
# spatial join facilities and admin 2 areas
lbr_geo_filt = lbr_geo_filt.to_crs(adm2.crs)
facilities = gpd.sjoin(lbr_geo_filt, adm2[['OBJECTID', 'WB_ADM2_CO', 'geometry']], how='left')

In [168]:
lbr_geo_filt

,admin_county,admin_district,today,latitude,longitude,_getcoordinate_altitude,precision,facilitytype,ownership,opendate,functional,offer_anc,offer_obstetric,offer_emonc,offer_postpartum,offer_newborncare,offer_maternalnewboen,offer_csection,offer_childimmune,offer_imci,offer_essentialnutrition,offer_fpcouple,offer_fpcounsel,offer_adolescentsrhr,offer_sexgender,offer_detectmanage,offer_precenttreat,dup,update_date,geocoding_method,facility,geometry,bool_1_100_offer_anc,bool_2_100_offer_anc,bool_3_100_offer_anc,bool_4_100_offer_anc,bool_1_2000_offer_anc,bool_2_2000_offer_anc,bool_3_2000_offer_anc,bool_4_2000_offer_anc,bool_1_100_offer_emonc,bool_2_100_offer_emonc,bool_3_100_offer_emonc,bool_4_100_offer_emonc,bool_1_2000_offer_emonc,bool_2_2000_offer_emonc,bool_3_2000_offer_emonc,bool_4_2000_offer_emonc,bool_1_100_offer_childimmune,bool_2_100_offer_childimmune,bool_3_100_offer_childimmune,bool_4_100_offer_childimmune,bool_1_2000_offer_childimmune,bool_2_2000_offer_childimmune,bool_3_2000_offer_childimmune,bool_4_2000_offer_childimmune,bool_1_100_offer_essentialnutrition,bool_2_100_offer_essentialnutrition,bool_3_100_offer_essentialnutrition,bool_4_100_offer_essentialnutrition,bool_1_2000_offer_essentialnutrition,bool_2_2000_offer_essentialnutrition,bool_3_2000_offer_essentialnutrition,bool_4_2000_offer_essentialnutrition,bool_1_100_offer_adolescentsrhr,bool_2_100_offer_adolescentsrhr,bool_3_100_offer_adolescentsrhr,bool_4_100_offer_adolescentsrhr,bool_1_2000_offer_adolescentsrhr,bool_2_2000_offer_adolescentsrhr,bool_3_2000_offer_adolescentsrhr,bool_4_2000_offer_adolescentsrhr,bool_1_100,bool_2_100,bool_3_100,bool_4_100,bool_1_2000,bool_2_2000,bool_3_2000,bool_4_2000
0,Montserrado,Somalia Drive,2021-08-07,6.331593,-10.743366,4.009424,4.983,clinic,Private (non-faith based),2013,functional,1.0,1.0,0.0,1.0,1.0,1.0,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0,07aug2021,NaN,NaN,POINT (-10.74337 6.33159),True,True,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,True,True,True,True,True
1,Montserrado,Central Monrovia,2021-10-19,6.304997,-10.799709,2.900000,4.978,clinic,"Private, (faith based)",1988,functional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,19oct2021,NaN,NaN,POINT (-10.79971 6.30500),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True
2,Montserrado,St Paul River,2021-11-18,6.412980,-10.778231,-2.712283,4.933,clinic,Private (non-faith based),2018,functional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,18nov2021,NaN,NaN,POINT (-10.77823 6.41298),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True
3,Montserrado,St Paul River,2021-08-23,6.413318,-10.778419,17.595896,4.966,clinic,Public,1950,functional,1.0,0.0,0.0,1.0,1.0,1.0,NaN,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0,23aug2021,NaN,NaN,POINT (-10.77842 6.41332),False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True,False,False,True,True,True,True,True,True
4,Montserrado,Commonwealth,2021-11-17,6.311818,-10.654602,22.641799,4.950,clinic,"Private, (faith based)",2013,functional,1.0,1.0,0.0,0.0,1.0,1.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,17nov2021,NaN,NaN,POINT (-10.65460 6.31182),False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [169]:
lbr_geo_filt

,admin_county,admin_district,today,latitude,longitude,_getcoordinate_altitude,precision,facilitytype,ownership,opendate,functional,offer_anc,offer_obstetric,offer_emonc,offer_postpartum,offer_newborncare,offer_maternalnewboen,offer_csection,offer_childimmune,offer_imci,offer_essentialnutrition,offer_fpcouple,offer_fpcounsel,offer_adolescentsrhr,offer_sexgender,offer_detectmanage,offer_precenttreat,dup,update_date,geocoding_method,facility,geometry,bool_1_100_offer_anc,bool_2_100_offer_anc,bool_3_100_offer_anc,bool_4_100_offer_anc,bool_1_2000_offer_anc,bool_2_2000_offer_anc,bool_3_2000_offer_anc,bool_4_2000_offer_anc,bool_1_100_offer_emonc,bool_2_100_offer_emonc,bool_3_100_offer_emonc,bool_4_100_offer_emonc,bool_1_2000_offer_emonc,bool_2_2000_offer_emonc,bool_3_2000_offer_emonc,bool_4_2000_offer_emonc,bool_1_100_offer_childimmune,bool_2_100_offer_childimmune,bool_3_100_offer_childimmune,bool_4_100_offer_childimmune,bool_1_2000_offer_childimmune,bool_2_2000_offer_childimmune,bool_3_2000_offer_childimmune,bool_4_2000_offer_childimmune,bool_1_100_offer_essentialnutrition,bool_2_100_offer_essentialnutrition,bool_3_100_offer_essentialnutrition,bool_4_100_offer_essentialnutrition,bool_1_2000_offer_essentialnutrition,bool_2_2000_offer_essentialnutrition,bool_3_2000_offer_essentialnutrition,bool_4_2000_offer_essentialnutrition,bool_1_100_offer_adolescentsrhr,bool_2_100_offer_adolescentsrhr,bool_3_100_offer_adolescentsrhr,bool_4_100_offer_adolescentsrhr,bool_1_2000_offer_adolescentsrhr,bool_2_2000_offer_adolescentsrhr,bool_3_2000_offer_adolescentsrhr,bool_4_2000_offer_adolescentsrhr,bool_1_100,bool_2_100,bool_3_100,bool_4_100,bool_1_2000,bool_2_2000,bool_3_2000,bool_4_2000
0,Montserrado,Somalia Drive,2021-08-07,6.331593,-10.743366,4.009424,4.983,clinic,Private (non-faith based),2013,functional,1.0,1.0,0.0,1.0,1.0,1.0,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0,07aug2021,NaN,NaN,POINT (-10.74337 6.33159),True,True,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,True,True,True,True,True
1,Montserrado,Central Monrovia,2021-10-19,6.304997,-10.799709,2.900000,4.978,clinic,"Private, (faith based)",1988,functional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,19oct2021,NaN,NaN,POINT (-10.79971 6.30500),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True
2,Montserrado,St Paul River,2021-11-18,6.412980,-10.778231,-2.712283,4.933,clinic,Private (non-faith based),2018,functional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,18nov2021,NaN,NaN,POINT (-10.77823 6.41298),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True
3,Montserrado,St Paul River,2021-08-23,6.413318,-10.778419,17.595896,4.966,clinic,Public,1950,functional,1.0,0.0,0.0,1.0,1.0,1.0,NaN,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0,23aug2021,NaN,NaN,POINT (-10.77842 6.41332),False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True,False,False,True,True,True,True,True,True
4,Montserrado,Commonwealth,2021-11-17,6.311818,-10.654602,22.641799,4.950,clinic,"Private, (faith based)",2013,functional,1.0,1.0,0.0,0.0,1.0,1.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,17nov2021,NaN,NaN,POINT (-10.65460 6.31182),False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [182]:
for x in range(1,5):
    print(x)

1
2
3
4


In [196]:
def calc_percentages_within_buffer_of_roads(roads, buffer, road_category, sub_category = None):
    #res_copy = res.copy()
    #for buffer in buffers:
    #facilities[['bool_1_100','bool_2_100','bool_3_100','bool_4_100','WB_ADM2_CO']].groupby('WB_ADM2_CO').sum()
    
    if sub_category is not None:
        query = "bool_"+ str(road_category) + "_"+str(buffer)+"_"+sub_category
    else:
        query = "bool_"+ str(road_category) + "_"+str(buffer)

    print(query)
    facilities_copy = facilities.copy()
    filtered_df = facilities_copy[facilities_copy[query].notnull()]
    #filtered_df[query] = filtered_df[query].astype(int)
    filtered_df[query] = filtered_df.loc[:, query].astype(int)
    sum_within = filtered_df[[query,'WB_ADM2_CO']].groupby('WB_ADM2_CO').sum()
    count_within = filtered_df[[query,'WB_ADM2_CO']].groupby('WB_ADM2_CO').count()
    #count_within = filtered_df[[query,'WB_ADM2_CO']].groupby('WB_ADM2_CO').count().rename(columns={query:'count'})
    #percent_within = filtered_df / count_within
    #percent_within = sum_within.apply(lambda x: x/count_within['count'])
    percent_within = sum_within / count_within
    to_join = sum_within.join(count_within, rsuffix="_count").join(percent_within, rsuffix="_pct")
    print(query)
    #print(sum_within)
    #print(count_within)
    #print(percent_within)
    print(len(to_join))
    #print(to_join)

    #res_copy = res_copy.merge(to_join, left_on="WB_ADM2_CO", right_index=True)
    return to_join

In [197]:
pd.set_option('display.max_columns', None)

In [198]:
res_copy = res.copy()

In [199]:
for buffer in [100,2000]:
    # do for all facilities
    for x in range(1,5):
        to_join = calc_percentages_within_buffer_of_roads(roads, buffer, x, sub_category=None)
        res_copy = res_copy.merge(to_join, how='left', left_on="WB_ADM2_CO", right_on="WB_ADM2_CO")
    for sub_category in sub_category_list:
        for x in range(1,5):
            to_join = calc_percentages_within_buffer_of_roads(roads, buffer, x, sub_category=sub_category)
            res_copy = res_copy.merge(to_join, how='left', left_on="WB_ADM2_CO", right_on="WB_ADM2_CO")

bool_1_100
bool_1_100
64
bool_2_100
bool_2_100
64
bool_3_100
bool_3_100
64
bool_4_100
bool_4_100
64
bool_1_100_offer_anc
bool_1_100_offer_anc
64
bool_2_100_offer_anc
bool_2_100_offer_anc
64
bool_3_100_offer_anc
bool_3_100_offer_anc
64
bool_4_100_offer_anc
bool_4_100_offer_anc
64
bool_1_100_offer_emonc
bool_1_100_offer_emonc
59
bool_2_100_offer_emonc
bool_2_100_offer_emonc
59
bool_3_100_offer_emonc
bool_3_100_offer_emonc
59
bool_4_100_offer_emonc
bool_4_100_offer_emonc
59
bool_1_100_offer_childimmune
bool_1_100_offer_childimmune
64
bool_2_100_offer_childimmune
bool_2_100_offer_childimmune
64
bool_3_100_offer_childimmune
bool_3_100_offer_childimmune
64
bool_4_100_offer_childimmune
bool_4_100_offer_childimmune
64
bool_1_100_offer_essentialnutrition
bool_1_100_offer_essentialnutrition
64


/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

bool_2_100_offer_essentialnutrition
bool_2_100_offer_essentialnutrition
64
bool_3_100_offer_essentialnutrition
bool_3_100_offer_essentialnutrition
64
bool_4_100_offer_essentialnutrition
bool_4_100_offer_essentialnutrition
64
bool_1_100_offer_adolescentsrhr
bool_1_100_offer_adolescentsrhr
64
bool_2_100_offer_adolescentsrhr
bool_2_100_offer_adolescentsrhr
64
bool_3_100_offer_adolescentsrhr
bool_3_100_offer_adolescentsrhr
64
bool_4_100_offer_adolescentsrhr
bool_4_100_offer_adolescentsrhr
64
bool_1_2000
bool_1_2000
64
bool_2_2000
bool_2_2000
64
bool_3_2000
bool_3_2000
64
bool_4_2000
bool_4_2000
64
bool_1_2000_offer_anc
bool_1_2000_offer_anc
64
bool_2_2000_offer_anc
bool_2_2000_offer_anc
64
bool_3_2000_offer_anc
bool_3_2000_offer_anc
64
bool_4_2000_offer_anc
bool_4_2000_offer_anc
64
bool_1_2000_offer_emonc


/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

bool_1_2000_offer_emonc
59
bool_2_2000_offer_emonc
bool_2_2000_offer_emonc
59
bool_3_2000_offer_emonc
bool_3_2000_offer_emonc
59
bool_4_2000_offer_emonc
bool_4_2000_offer_emonc
59
bool_1_2000_offer_childimmune
bool_1_2000_offer_childimmune
64
bool_2_2000_offer_childimmune
bool_2_2000_offer_childimmune
64
bool_3_2000_offer_childimmune
bool_3_2000_offer_childimmune
64
bool_4_2000_offer_childimmune
bool_4_2000_offer_childimmune
64
bool_1_2000_offer_essentialnutrition
bool_1_2000_offer_essentialnutrition
64
bool_2_2000_offer_essentialnutrition
bool_2_2000_offer_essentialnutrition
64
bool_3_2000_offer_essentialnutrition
bool_3_2000_offer_essentialnutrition
64
bool_4_2000_offer_essentialnutrition
bool_4_2000_offer_essentialnutrition
64
bool_1_2000_offer_adolescentsrhr
bool_1_2000_offer_adolescentsrhr
64
bool_2_2000_offer_adolescentsrhr
bool_2_2000_offer_adolescentsrhr
64
bool_3_2000_offer_adolescentsrhr
bool_3_2000_offer_adolescentsrhr
64
bool_4_2000_offer_adolescentsrhr
bool_4_2000_offer_ad

/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [200]:
len(res_copy)

64

In [201]:
#res_copy.to_file(os.path.join(out_folder, "health_roads_results_sub_categories3.shp"), driver="ESRI Shapefile")

In [202]:
res_formatted = res_copy.copy()

In [203]:
res_formatted = res_formatted.drop(columns=['Shape_Leng', 'Shape_Area', 'geometry'])

In [204]:
res_formatted

,index,OBJECTID,ISO_A2,WB_ADM1_CO,WB_ADM0_CO,WB_ADM0_NA,WB_ADM1_NA,WB_ADM2_CO,WB_ADM2_NA,ISO3,pop,pop_120_health,health_pct,pop_120_hospital,hospital_pct,pop_120_offer_anc,offer_anc_pct,pop_120_offer_emonc,offer_emonc_pct,pop_120_offer_childimmune,offer_childimmune_pct,pop_120_offer_essentialnutrition,offer_essentialnutrition_pct,pop_120_offer_adolescentsrhr,offer_adolescentsrhr_pct,bool_1_100,bool_1_100_count,bool_1_100_pct,bool_2_100,bool_2_100_count,bool_2_100_pct,bool_3_100,bool_3_100_count,bool_3_100_pct,bool_4_100,bool_4_100_count,bool_4_100_pct,bool_1_100_offer_anc,bool_1_100_offer_anc_count,bool_1_100_offer_anc_pct,bool_2_100_offer_anc,bool_2_100_offer_anc_count,bool_2_100_offer_anc_pct,bool_3_100_offer_anc,bool_3_100_offer_anc_count,bool_3_100_offer_anc_pct,bool_4_100_offer_anc,bool_4_100_offer_anc_count,bool_4_100_offer_anc_pct,bool_1_100_offer_emonc,bool_1_100_offer_emonc_count,bool_1_100_offer_emonc_pct,bool_2_100_offer_emonc,bool_2_100_offer_emonc_count,bool_2_100_offer_emonc_pct,bool_3_100_offer_emonc,bool_3_100_offer_emonc_count,bool_3_100_offer_emonc_pct,bool_4_100_offer_emonc,bool_4_100_offer_emonc_count,bool_4_100_offer_emonc_pct,bool_1_100_offer_childimmune,bool_1_100_offer_childimmune_count,bool_1_100_offer_childimmune_pct,bool_2_100_offer_childimmune,bool_2_100_offer_childimmune_count,bool_2_100_offer_childimmune_pct,bool_3_100_offer_childimmune,bool_3_100_offer_childimmune_count,bool_3_100_offer_childimmune_pct,bool_4_100_offer_childimmune,bool_4_100_offer_childimmune_count,bool_4_100_offer_childimmune_pct,bool_1_100_offer_essentialnutrition,bool_1_100_offer_essentialnutrition_count,bool_1_100_offer_essentialnutrition_pct,bool_2_100_offer_essentialnutrition,bool_2_100_offer_essentialnutrition_count,bool_2_100_offer_essentialnutrition_pct,bool_3_100_offer_essentialnutrition,bool_3_100_offer_essentialnutrition_count,bool_3_100_offer_essentialnutrition_pct,bool_4_100_offer_essentialnutrition,bool_4_100_offer_essentialnutrition_count,bool_4_100_offer_essentialnutrition_pct,bool_1_100_offer_adolescentsrhr,bool_1_100_offer_adolescentsrhr_count,bool_1_100_offer_adolescentsrhr_pct,bool_2_100_offer_adolescentsrhr,bool_2_100_offer_adolescentsrhr_count,bool_2_100_offer_adolescentsrhr_pct,bool_3_100_offer_adolescentsrhr,bool_3_100_offer_adolescentsrhr_count,bool_3_100_offer_adolescentsrhr_pct,bool_4_100_offer_adolescentsrhr,bool_4_100_offer_adolescentsrhr_count,bool_4_100_offer_adolescentsrhr_pct,bool_1_2000,bool_1_2000_count,bool_1_2000_pct,bool_2_2000,bool_2_2000_count,bool_2_2000_pct,bool_3_2000,bool_3_2000_count,bool_3_2000_pct,bool_4_2000,bool_4_2000_count,bool_4_2000_pct,bool_1_2000_offer_anc,bool_1_2000_offer_anc_count,bool_1_2000_offer_anc_pct,bool_2_2000_offer_anc,bool_2_2000_offer_anc_count,bool_2_2000_offer_anc_pct,bool_3_2000_offer_anc,bool_3_2000_offer_anc_count,bool_3_2000_offer_anc_pct,bool_4_2000_offer_anc,bool_4_2000_offer_anc_count,bool_4_2000_offer_anc_pct,bool_1_2000_offer_emonc,bool_1_2000_offer_emonc_count,bool_1_2000_offer_emonc_pct,bool_2_2000_offer_emonc,bool_2_2000_offer_emonc_count,bool_2_2000_offer_emonc_pct,bool_3_2000_offer_emonc,bool_3_2000_offer_emonc_count,bool_3_2000_offer_emonc_pct,bool_4_2000_offer_emonc,bool_4_2000_offer_emonc_count,bool_4_2000_offer_emonc_pct,bool_1_2000_offer_childimmune,bool_1_2000_offer_childimmune_count,bool_1_2000_offer_childimmune_pct,bool_2_2000_offer_childimmune,bool_2_2000_offer_childimmune_count,bool_2_2000_offer_childimmune_pct,bool_3_2000_offer_childimmune,bool_3_2000_offer_childimmune_count,bool_3_2000_offer_childimmune_pct,bool_4_2000_offer_childimmune,bool_4_2000_offer_childimmune_count,bool_4_2000_offer_childimmune_pct,bool_1_2000_offer_essentialnutrition,bool_1_2000_offer_essentialnutrition_count,bool_1_2000_offer_essentialnutrition_pct,bool_2_2000_offer_essentialnutrition,bool_2_2000_offer_essentialnutrition_count,bool_2_2000_offer_essentialnutrition_pct,bool_3_2000_offer_essentialnutrition,bool_3_2000_offer_essentialnutrition_count,bool_3_2000_

In [205]:
cols = res_formatted.columns.tolist()
cols

['index',
 'OBJECTID',
 'ISO_A2',
 'WB_ADM1_CO',
 'WB_ADM0_CO',
 'WB_ADM0_NA',
 'WB_ADM1_NA',
 'WB_ADM2_CO',
 'WB_ADM2_NA',
 'ISO3',
 'pop',
 'pop_120_health',
 'health_pct',
 'pop_120_hospital',
 'hospital_pct',
 'pop_120_offer_anc',
 'offer_anc_pct',
 'pop_120_offer_emonc',
 'offer_emonc_pct',
 'pop_120_offer_childimmune',
 'offer_childimmune_pct',
 'pop_120_offer_essentialnutrition',
 'offer_essentialnutrition_pct',
 'pop_120_offer_adolescentsrhr',
 'offer_adolescentsrhr_pct',
 'bool_1_100',
 'bool_1_100_count',
 'bool_1_100_pct',
 'bool_2_100',
 'bool_2_100_count',
 'bool_2_100_pct',
 'bool_3_100',
 'bool_3_100_count',
 'bool_3_100_pct',
 'bool_4_100',
 'bool_4_100_count',
 'bool_4_100_pct',
 'bool_1_100_offer_anc',
 'bool_1_100_offer_anc_count',
 'bool_1_100_offer_anc_pct',
 'bool_2_100_offer_anc',
 'bool_2_100_offer_anc_count',
 'bool_2_100_offer_anc_pct',
 'bool_3_100_offer_anc',
 'bool_3_100_offer_anc_count',
 'bool_3_100_offer_anc_pct',
 'bool_4_100_offer_anc',
 'bool_4_100_of

In [210]:
cols2 = [
 'OBJECTID',
 'ISO_A2',
 'WB_ADM1_CO',
 'WB_ADM0_CO',
 'WB_ADM0_NA',
 'WB_ADM1_NA',
 'WB_ADM2_CO',
 'WB_ADM2_NA',
 'ISO3',
 'pop',
 'pop_120_health',
 'pop_120_hospital',
 'health_pct',
 'hospital_pct',
 'bool_1_100_count',
 'bool_1_2000',
 'bool_2_2000',
 'bool_3_2000',
 'bool_4_2000',
 'bool_1_2000_pct',
 'bool_2_2000_pct',
 'bool_3_2000_pct',
 'bool_4_2000_pct',
 'bool_1_100',
 'bool_2_100',
 'bool_3_100',
 'bool_4_100',
 'bool_1_100_pct',
 'bool_2_100_pct',
 'bool_3_100_pct',
 'bool_4_100_pct',
 ]

In [212]:
res_formatted2 = res_formatted[cols2]
res_formatted2

,OBJECTID,ISO_A2,WB_ADM1_CO,WB_ADM0_CO,WB_ADM0_NA,WB_ADM1_NA,WB_ADM2_CO,WB_ADM2_NA,ISO3,pop,pop_120_health,pop_120_hospital,health_pct,hospital_pct,bool_1_100_count,bool_1_2000,bool_2_2000,bool_3_2000,bool_4_2000,bool_1_2000_pct,bool_2_2000_pct,bool_3_2000_pct,bool_4_2000_pct,bool_1_100,bool_2_100,bool_3_100,bool_4_100,bool_1_100_pct,bool_2_100_pct,bool_3_100_pct,bool_4_100_pct
0,22304,LR,1814,144,Liberia,Bomi,39172,Klay,LBR,141868.437500,141868.437500,141802.968750,1.000000,0.999539,32,16,26,32,32,0.500000,0.812500,1.0,1.0,6,12,24,25,0.187500,0.375000,0.750000,0.781250
1,22305,LR,1814,144,Liberia,Bomi,39173,Mecca,LBR,11020.040039,11020.040039,11020.040039,1.000000,1.000000,2,2,2,2,2,1.000000,1.000000,1.0,1.0,2,2,2,2,1.000000,1.000000,1.000000,1.000000
2,22306,LR,1815,144,Liberia,Bong,39174,Fuamah,LBR,38777.539062,34315.687500,33150.644531,0.884937,0.854893,3,0,1,3,3,0.000000,0.333333,1.0,1.0,0,0,1,2,0.000000,0.000000,0.333333,0.666667
3,22307,LR,1815,144,Liberia,Bong,39175,Jorquelleh,LBR,88549.031250,88312.546875,86918.000000,0.997329,0.981580,14,13,13,14,14,0.928571,0.928571,1.0,1.0,6,6,14,14,0.428571,0.428571,1.000000,1.000000
4,22308,LR,1815,144,Liberia,Bong,39176,Kokoyah,LBR,28408.078125,28299.513672,24386.886719,0.996178,0.858449,5,4,5,5,5,0.800000,1.000000,1.0,1.0,4,5,5,5,0.800000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,22363,LR,1828,144,Liberia,Sinoe,39231,Greenville,LBR,22259.300781,22259.300781,22187.615234,1.000000,0.996780,4,0,4,4,4,0.000000,1.000000,1.0,1.0,0,2,4,4,0.000000,0.500000,1.000000,1.000000
60,22364,LR,1828,144,Liberia,Sinoe,39232,Jaedae Jaedepo,LBR,22469.167969,8966.367188,5775.648926,0.399052,0.257048,5,0,3,5,5,0.000000,0.600000,1.0,1.0,0,1,5,5,0.000000,0.200000,1.000000,1.000000
61,22365,LR,1828,144,Liberia,Sinoe,39233,Juarzon,LBR,25811.957031,17730.841797,14539.828125,0.686924,0.563298,7,3,4,7,7,0.428571,0.571429,1.0,1.0,0,1,6,6,0.000000,0.142857,0.857143,0.857143
62,22366,LR,1828,144,Liberia,Sinoe,39234,Kpayan,LBR,10879.102539,9507.745117,8430.095703,0.873946,0.774889,4,0,3,4,4,0.000000,0.750000,1.0,1.0,0,2,4,4,0.000000,0.500000,1.000000,1.000000


In [213]:
# also export as csv
res_formatted2.to_csv(os.path.join(out_folder, "statistics_liberia_sub_categories.csv"))

In [51]:
#res_osmlr = facilities[['bool_1','bool_2','bool_3','bool_4','WB_ADM2_CO']].groupby('WB_ADM2_CO').sum()
#res_osmlr = facilities[['bool_1_100','bool_2_100','bool_3_100','bool_4_100','bool_1_2000','bool_2_2000','bool_3_2000','bool_4_2000','WB_ADM2_CO']].groupby('WB_ADM2_CO').sum()

# res_osmlr_100 = facilities[['bool_1_100','bool_2_100','bool_3_100','bool_4_100','WB_ADM2_CO']].groupby('WB_ADM2_CO').sum()
# res_osmlr_2000 = facilities[['bool_1_2000','bool_2_2000','bool_3_2000','bool_4_2000','WB_ADM2_CO']].groupby('WB_ADM2_CO').sum()

# res_count_100 = facilities[['WB_ADM2_CO','bool_1_100']].groupby('WB_ADM2_CO').count().rename(columns={'bool_1_100':'count_100'})
# res_count_2000 = facilities[['WB_ADM2_CO','bool_1_2000']].groupby('WB_ADM2_CO').count().rename(columns={'bool_1_2000':'count_2000'})

# res_osmlr_pct_100 = res_osmlr_100.apply(lambda x: x/res_count_100['count_100'])
# res_osmlr_pct_2000 = res_osmlr_2000.apply(lambda x: x/res_count_2000['count_2000'])

# res_osmlr_100 = res_osmlr_100.join(res_count_100)
# res_osmlr_2000 = res_osmlr_2000.join(res_count_2000)

# res_osmlr_100 = res_osmlr_100.join(res_osmlr_pct_100, rsuffix="_pct")
# res_osmlr_2000 = res_osmlr_2000.join(res_osmlr_pct_2000, rsuffix="_pct")

# res_merge_100 = res.merge(res_osmlr_100, left_on="WB_ADM2_CO", right_index=True)
# res_merge_2000 = res.merge(res_osmlr_2000, left_on="WB_ADM2_CO", right_index=True)

In [52]:
#res_osmlr_100

In [53]:
#res_merge_100.to_file(os.path.join(out_folder, "health_roads_2nd_rd_100.shp"), driver="ESRI Shapefile")

In [54]:
#res_merge_2000.to_file(os.path.join(out_folder, "health_roads_2nd_rd_2000.shp"), driver="ESRI Shapefile")

### FB Analysis

In [55]:
input_dir

'/home/public/Data/PROJECTS/Health'

In [56]:
fb = pd.read_csv(os.path.join(input_dir, 'facebook', 'lbr_relative_wealth_index.csv'))

In [57]:
fb_geoms = [Point(xy) for xy in zip(fb.longitude, fb.latitude)]
fb_geo = gpd.GeoDataFrame(fb, crs='EPSG:4326', geometry=fb_geoms)

### Calculating zonal stats of travel times within FB grids cells

In [58]:
tt_health_rio = rio.open(os.path.join(out_folder, "tt_health_2nd_rd.tif"))
tt_health = tt_health_rio.read(1, masked=True)
tt_hospital_rio = rio.open(os.path.join(out_folder, "tt_hospital_2nd_rd.tif"))
tt_hospital = tt_hospital_rio.read(1, masked=True)

In [59]:
fb_zs = pd.DataFrame(zonal_stats(fb_geo, tt_health.filled(), affine=tt_health_rio.transform, stats='mean', nodata=tt_health_rio.nodata)).rename(columns={'mean':'tt_health'})
fb_zs_hosp = pd.DataFrame(zonal_stats(fb_geo, tt_hospital.filled(), affine=tt_hospital_rio.transform, stats='mean', nodata=tt_hospital_rio.nodata)).rename(columns={'mean':'tt_hospital'})

In [60]:
fb_geo = fb_geo.join(fb_zs).join(fb_zs_hosp)

In [61]:
fb_geo

,latitude,longitude,rwi,error,geometry,tt_health,tt_hospital
0,4.576425,-7.987061,-0.902,0.295,POINT (-7.98706 4.57642),87.080078,185.625931
1,6.217012,-10.228271,-0.082,0.289,POINT (-10.22827 6.21701),7.249699,57.827938
2,5.626919,-8.316650,-0.517,0.286,POINT (-8.31665 5.62692),33.827759,146.152924
3,6.544560,-9.656982,-0.489,0.267,POINT (-9.65698 6.54456),38.609650,107.302048
4,7.852499,-9.656982,-0.761,0.260,POINT (-9.65698 7.85250),46.817535,74.537605
...,...,...,...,...,...,...,...
6184,7.111795,-8.822021,-0.277,0.271,POINT (-8.82202 7.11179),5.794456,33.257629
6185,6.238855,-9.547119,-0.942,0.285,POINT (-9.54712 6.23886),161.502319,215.188629
6186,6.369894,-9.964600,-0.279,0.269,POINT (-9.96460 6.36989),40.256237,89.289223
6187,6.806444,-9.613037,-0.457,0.268,POINT (-9.61304 6.80644),15.449747,52.030640


In [62]:
fb_geo.loc[:, "rwi_cut"] = pd.qcut(fb_geo['rwi'], [0, .2, .4, .6, .8, 1.], labels=['lowest', 'second-lowest', 'middle', 'second-highest', 'highest'])

In [63]:
# fb_geo.groupby()

In [64]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from pyquadkey2 import quadkey

from pyquadkey2.quadkey import QuadKey
from pyquadkey2.quadkey import TileAnchor, QuadKey
from shapely.geometry import LineString

In [65]:
fb_geo = gpd.sjoin(fb_geo, adm2[['WB_ADM2_CO', 'WB_ADM2_NA', 'geometry']])

In [66]:
# not sure why this doesn't work aint nobody got time for that
# seems like the spatial join above accomplished that
def get_point_in_polygon(lat, lon, polygons):
    """ 
    @param lat: double
    @param lon: double
    @param polygons: dict
    @return geo_id: str
    """
    point = Point(lon, lat)
    for geo_id in polygons:
        polygon = polygons[geo_id]
        if polygon.contains(point):
            return geo_id
        else:
            return 'null'

shapefile = adm2.copy()
polygons = dict(zip(shapefile['WB_ADM2_CO'], shapefile['geometry']))
print(shapefile.shape)
shapefile.head()

fb_geo['geo_id'] = fb_geo.apply(lambda x: get_point_in_polygon(x['latitude'], x['longitude'], polygons), axis=1)

(64, 13)


In [67]:
fb_geo

,latitude,longitude,rwi,error,geometry,tt_health,tt_hospital,rwi_cut,index_right,WB_ADM2_CO,WB_ADM2_NA,geo_id
0,4.576425,-7.987061,-0.902,0.295,POINT (-7.98706 4.57642),87.080078,185.625931,lowest,29,39201,Trehn,null
40,4.598327,-7.921143,-0.190,0.279,POINT (-7.92114 4.59833),4.000000,84.278542,second-highest,29,39201,Trehn,null
127,4.598327,-7.811279,-0.628,0.288,POINT (-7.81128 4.59833),15.812191,54.465435,lowest,29,39201,Trehn,null
141,4.685929,-7.899170,-0.403,0.278,POINT (-7.89917 4.68593),25.216219,87.048248,middle,29,39201,Trehn,null
165,4.554522,-7.921143,-0.491,0.289,POINT (-7.92114 4.55452),48.523163,101.594742,second-lowest,29,39201,Trehn,null
...,...,...,...,...,...,...,...,...,...,...,...,...
4323,6.740986,-11.348877,-0.222,0.325,POINT (-11.34888 6.74099),26.932196,26.932196,second-highest,22,39194,Robertsport,null
4489,6.740986,-11.370850,-0.500,0.353,POINT (-11.37085 6.74099),28.139103,28.139103,second-lowest,22,39194,Robertsport,null
4737,6.762806,-11.326904,-0.137,0.287,POINT (-11.32690 6.76281),3.182249,40.285004,highest,22,39194,Robertsport,null
5654,6.740986,-11.326904,-0.032,0.291,POINT (-11.32690 6.74099),19.202011,19.974154,highest,22,39194,Robertsport,null


In [68]:
population = pd.read_csv(os.path.join(input_dir, 'facebook', 'population_lbr_2019-07-01.csv'))

In [69]:
population = population.rename(columns={'Population': 'pop_2020'})

In [70]:
population['quadkey'] = population.apply(lambda x: str(quadkey.from_geo((x['Lat'], x['Lon']), 14)), axis=1)

In [71]:
# change quadkey format to str
population["quadkey1"] = population["quadkey"].astype('str')
# fill 13 digit quadkeys with 0 before the QK
#population['quadkey1'] = population['quadkey1'].apply(lambda x: x.zfill(14))
# take column with quadkeys and transform it to list and then to QuadKey object
fb_qk = population["quadkey1"].tolist()
FB_QK = [QuadKey(i) for i in fb_qk]
# locate the four corners of QuadKey tiles
SW = []
NE = []
SE = []
NW = []

for i in FB_QK:
    #south west point
    sw = i.to_geo(anchor=TileAnchor.ANCHOR_SW)
    SW.append(sw) 
    #north west point
    ne = i.to_geo(anchor=TileAnchor.ANCHOR_NE)
    NE.append(ne) 
    #south east point 
    se = i.to_geo(anchor=TileAnchor.ANCHOR_SE)
    SE.append(se)
    #north west point 
    nw = i.to_geo(anchor=TileAnchor.ANCHOR_NW)
    NW.append(nw)
# flip lat and lon for tile corner coordinates
swFinal = [(i[1], i[0]) for i in SW]
neFinal = [(i[1], i[0]) for i in NE]
seFinal = [(i[1], i[0]) for i in SE]
nwFinal = [(i[1], i[0]) for i in NW]
# generate Polygon object based on four corner coordinates
tiledata = [Polygon([nw, sw, se, ne]) for sw, ne, se, nw in zip(swFinal, neFinal, seFinal, nwFinal)]
# create GeoDataFrame with Polygons
crs = ("epsg:4326")
gdf = gpd.GeoDataFrame(geometry = tiledata, crs = crs)

In [72]:
# add QuadKeys to gdf
gdf['quadkey'] = FB_QK
# change type of quadkey column to str
gdf["quadkey"] = gdf["quadkey"].astype('str')
# merge gdf and fb data
gdf = gdf.merge(population, left_on = 'quadkey', right_on = 'quadkey1', how = 'inner')

In [73]:
gdf

,geometry,quadkey_x,Lat,Lon,pop_2020,quadkey_y,quadkey1
0,"POLYGON ((-10.39307 6.55547, -10.39307 6.53365...",03333022120313,6.552083,-10.371528,17.792957,03333022120313,03333022120313
1,"POLYGON ((-10.39307 6.55547, -10.39307 6.53365...",03333022120313,6.551528,-10.371250,17.792957,03333022120313,03333022120313
2,"POLYGON ((-10.39307 6.55547, -10.39307 6.53365...",03333022120313,6.552083,-10.371250,17.792957,03333022120313,03333022120313
3,"POLYGON ((-10.39307 6.55547, -10.39307 6.53365...",03333022120313,6.551806,-10.371250,17.792957,03333022120313,03333022120313
4,"POLYGON ((-10.39307 6.55547, -10.39307 6.53365...",03333022120313,6.544583,-10.382639,17.765105,03333022120313,03333022120313
...,...,...,...,...,...,...,...
360430527,"POLYGON ((-8.34961 7.23170, -8.34961 7.20990, ...",03333030220320,7.217361,-8.345139,10.509987,03333030220320,03333030220320
360430528,"POLYGON ((-8.34961 7.23170, -8.34961 7.20990, ...",03333030220320,7.211528,-8.346250,10.509987,03333030220320,03333030220320
360430529,"POLYGON ((-8.34961 7.23170, -8.34961 7.20990, ...",03333030220320,7.214861,-8.333472,10.509987,03333030220320,03333030220320
360430530,"POLYGON ((-8.34961 7.23170, -8.34961 7.20990, ...",03333030220320,7.217361,-8.338750,10.509987,03333030220320,03333030220320


In [75]:
sample_admin_gpd

,id,geometry
0,1,"POLYGON ((-10.68870 6.52571, -10.44156 6.52647..."


In [ ]:
# export to shapefile
#clip takes too long
#gdf_aoi = gpd.clip(gdf, sample_admin_gpd)

In [ ]:
gdf_aoi.to_file(os.path.join(out_folder, "test_quadkey_output.shp"))

In [ ]:
bing_tile_z14_pop = population.groupby('quadkey', as_index=False)['pop_2020'].sum()

In [ ]:
fb_geo['quadkey'] = fb_geo.apply(lambda x: str(quadkey.from_geo((x['latitude'], x['longitude']), 14)), axis=1)

In [ ]:
rwi = fb_geo.merge(bing_tile_z14_pop[['quadkey', 'pop_2020']], on='quadkey', how='inner')

In [ ]:
rwi.loc[:, "tt_health_bool"] = rwi.tt_health<=120
rwi.loc[:, "tt_hospital_bool"] = rwi.tt_hospital<=120

In [ ]:
rwi.columns

In [ ]:
rwi

In [ ]:
# rwi[['WB_ADM2_NA', 'tt_health_bool', 'rwi_cut', 'pop_2020']].groupby(['WB_ADM2_NA', 'tt_health_bool', 'rwi_cut']).sum()
# res_rwi = rwi[['WB_ADM2_NA', 'tt_health_bool', 'tt_hospital_bool', 'rwi_cut', 'pop_2020']].groupby(['WB_ADM2_NA', 'tt_health_bool', 'tt_hospital_bool', 'rwi_cut']).sum()

In [ ]:
rwi_pop = rwi[['WB_ADM2_NA', 'rwi_cut', 'pop_2020']].groupby(['WB_ADM2_NA', 'rwi_cut']).sum()

In [ ]:
rwi_health = rwi.loc[rwi.tt_health_bool==True, ['WB_ADM2_NA', 'rwi_cut', 'pop_2020']].groupby(['WB_ADM2_NA', 'rwi_cut']).sum().rename(columns={'pop_2020':'pop_120_health'})

In [ ]:
rwi_health

In [ ]:
rwi_hospital = rwi.loc[rwi.tt_hospital_bool==True, ['WB_ADM2_NA', 'rwi_cut', 'pop_2020']].groupby(['WB_ADM2_NA', 'rwi_cut']).sum().rename(columns={'pop_2020':'pop_120_hospital'})

In [ ]:
res_rwi = rwi_pop.join(rwi_health).join(rwi_hospital)

In [ ]:
res_rwi.loc[:, "health_pct"] = res_rwi['pop_120_health']/res_rwi['pop_2020']
res_rwi.loc[:, "hospital_pct"] = res_rwi['pop_120_hospital']/res_rwi['pop_2020']

In [ ]:
res_rwi.head()

In [ ]:
res_rwi.to_csv(os.path.join(out_folder, 'test_rwi2.csv'))

In [ ]:
rwi_pop_adm0 = rwi[['rwi_cut', 'pop_2020']].groupby(['rwi_cut']).sum()

In [ ]:
rwi_health_adm0 = rwi.loc[rwi.tt_health_bool==True, ['rwi_cut', 'pop_2020']].groupby(['rwi_cut']).sum().rename(columns={'pop_2020':'pop_120_health'})

In [ ]:
rwi_hospital_adm0 = rwi.loc[rwi.tt_hospital_bool==True, ['rwi_cut', 'pop_2020']].groupby(['rwi_cut']).sum().rename(columns={'pop_2020':'pop_120_hospital'})

In [ ]:
rwi_hospital_adm0

In [ ]:
res_rwi_adm0 = rwi_pop_adm0.join(rwi_health_adm0).join(rwi_hospital_adm0)

In [ ]:
res_rwi_adm0.loc[:, "health_pct"] = res_rwi_adm0['pop_120_health']/res_rwi_adm0['pop_2020']
res_rwi_adm0.loc[:, "hospital_pct"] = res_rwi_adm0['pop_120_hospital']/res_rwi_adm0['pop_2020']

In [ ]:
res_rwi_adm0

In [ ]:
res_rwi_adm0.to_csv(os.path.join(out_folder, 'rwi_adm0_round2.csv'))

# Additional stats

In [ ]:
# import stats gdf
res_gdf_import = gpd.read_file(os.path.join(out_folder, "results_gdf_w_sub_categories.shp"))

In [ ]:
res_gdf_import

In [ ]:
# check crs match
res_gdf_import.crs == adm2.crs

In [ ]:
adm2[:2]

In [ ]:
# spatial join facilities and admin 2 areas
res_gdf_import_w_admin2 = gpd.sjoin(res_gdf_import, adm2[['WB_ADM2_NA', 'WB_ADM2_CO', 'geometry']], how='left')

In [ ]:
res_gdf_import_w_admin2

In [ ]:
# spatial join facilities and FB stats
# should I create square polygons around the RWI centroids then do a spatial join?
#res_gdf_final = gpd.sjoin(res_gdf_import_w_admin2, adm2[['WB_ADM2_NA', 'WB_ADM2_CO', 'geometry']], how='left')